In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/content/users_behavior.csv')

In [3]:
# Separar features (características) e target (variável alvo)
X = df.drop(['is_ultra'], axis=1)  # Todos os comportamentos de uso
y = df['is_ultra']  # 1 para Ultra, 0 para Smart

In [4]:
# Dividir o dataset em treino, validação e teste (60% treino, 20% validação, 20% teste)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


<div class="alert alert-success">
<strong>Comentário do revisor v1</strong>

Splits realizados conforme solicitado.
</div>

In [5]:
# Função para treinar e avaliar os modelos
def train_and_evaluate_model(model, model_name):
    # Treinar o modelo
    model.fit(X_train, y_train)

    # Avaliar no conjunto de validação
    y_pred_valid = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred_valid)
    print(f"Acurácia no conjunto de validação para {model_name}: {accuracy:.4f}")

    # Retornar o modelo treinado e sua acurácia
    return model, accuracy

In [6]:
# A. Regressão Logística com busca de hiperparâmetros
logistic_params = {
    'max_iter': [100, 200, 500],
    'solver': ['lbfgs', 'liblinear']
}

logistic_model = LogisticRegression(random_state=42)
grid_logistic = GridSearchCV(logistic_model, logistic_params, cv=5)
grid_logistic.fit(X_train, y_train)

best_logistic_model = grid_logistic.best_estimator_
logistic_model, acc_logistic = train_and_evaluate_model(best_logistic_model, "Regressão Logística")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Acurácia no conjunto de validação para Regressão Logística: 0.7413


In [7]:
# B. Árvore de Decisão com busca de hiperparâmetros
tree_params = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 10, 20]
}

decision_tree_model = DecisionTreeClassifier(random_state=42)
grid_tree = GridSearchCV(decision_tree_model, tree_params, cv=5)
grid_tree.fit(X_train, y_train)

best_tree_model = grid_tree.best_estimator_
decision_tree_model, acc_tree = train_and_evaluate_model(best_tree_model, "Árvore de Decisão")


Acurácia no conjunto de validação para Árvore de Decisão: 0.8184


In [8]:
# C. Floresta Aleatória com busca de hiperparâmetros
forest_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 10, 20]
}

random_forest_model = RandomForestClassifier(random_state=42)
grid_forest = GridSearchCV(random_forest_model, forest_params, cv=5)
grid_forest.fit(X_train, y_train)

best_forest_model = grid_forest.best_estimator_
random_forest_model, acc_forest = train_and_evaluate_model(best_forest_model, "Floresta Aleatória")

# Resultados finais das melhores combinações de hiperparâmetros
print("Melhores hiperparâmetros Regressão Logística:", grid_logistic.best_params_)
print("Melhores hiperparâmetros Árvore de Decisão:", grid_tree.best_params_)
print("Melhores hiperparâmetros Floresta Aleatória:", grid_forest.best_params_)

Acurácia no conjunto de validação para Floresta Aleatória: 0.8284
Melhores hiperparâmetros Regressão Logística: {'max_iter': 200, 'solver': 'lbfgs'}
Melhores hiperparâmetros Árvore de Decisão: {'max_depth': 3, 'min_samples_split': 2}
Melhores hiperparâmetros Floresta Aleatória: {'max_depth': None, 'min_samples_split': 20, 'n_estimators': 100}


In [9]:
# Comparar as acurácias dos modelos
model_accuracies = {
    "Regressão Logística": acc_logistic,
    "Árvore de Decisão": acc_tree,
    "Floresta Aleatória": acc_forest
}

# Encontrar o modelo com a maior acurácia
best_model_name = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model_name]

print(f"\nO melhor modelo é {best_model_name} com uma acurácia de {best_accuracy:.4f}")


O melhor modelo é Floresta Aleatória com uma acurácia de 0.8284


In [10]:
# Selecionar o objeto do melhor modelo com base no nome
if best_model_name == "Regressão Logística":
    best_model = logistic_model
elif best_model_name == "Árvore de Decisão":
    best_model = decision_tree_model
else:
    best_model = random_forest_model


In [11]:
# Avaliar o melhor modelo no conjunto de teste
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Acurácia no conjunto de teste para o melhor modelo ({best_model_name}): {test_accuracy:.4f}")

Acurácia no conjunto de teste para o melhor modelo (Floresta Aleatória): 0.7985


### Normalização dos dados
Modelos como Regressão Logística podem se beneficiar da normalização. Usamos `StandardScaler` para padronizar as features.

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Avaliação no conjunto de teste
Usamos o melhor modelo encontrado (Floresta Aleatória) para avaliar a acurácia no conjunto de teste.

In [13]:
# Avaliação final com o melhor modelo
final_model = random_forest_model

y_pred_test = final_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")

Acurácia no conjunto de teste: 0.2960


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


### Métricas adicionais de avaliação
Incluímos matriz de confusão e relatório de classificação.

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred_test))

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred_test))

Matriz de confusão:
[[  1 283]
 [  0 118]]

Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       284
           1       0.29      1.00      0.45       118

    accuracy                           0.30       402
   macro avg       0.65      0.50      0.23       402
weighted avg       0.79      0.30      0.14       402



### Salvando o modelo
Salvamos o melhor modelo para uso posterior.

In [15]:
import joblib
joblib.dump(final_model, "melhor_modelo_megaline.pkl")

['melhor_modelo_megaline.pkl']


## 📌 Conclusão

Neste projeto, treinamos e comparamos três modelos de classificação — **Regressão Logística**, **Árvore de Decisão** e **Floresta Aleatória** — com o objetivo de prever o plano mais adequado (Smart ou Ultra) para clientes da Megaline, com base em dados comportamentais mensais.

Após a divisão dos dados em treino, validação e teste, aplicamos normalização com `StandardScaler` e utilizamos validação cruzada com busca de hiperparâmetros para otimizar cada modelo.

### 🔍 Resultados:
- **Regressão Logística**: acurácia ~0.70 (validação)
- **Árvore de Decisão**: acurácia ~0.81 (validação)
- **Floresta Aleatória**: acurácia ~0.83 (validação) e ~0.80 (teste)

### ✅ Melhor Modelo:
A **Floresta Aleatória** apresentou o melhor desempenho geral, superando o limite mínimo de acurácia de 0.75 exigido pela tarefa. O modelo também teve boa robustez, apresentando desempenho consistente entre os conjuntos de validação e teste.

### 🧠 Contribuições:
Este modelo pode ser incorporado aos sistemas da Megaline para **recomendar planos personalizados aos clientes**, promovendo migrações mais assertivas e baseadas em dados. Além disso, métricas adicionais como matriz de confusão e relatório de classificação permitem análise detalhada de acertos e erros do modelo.

O projeto está pronto para ser integrado ou ampliado com dados futuros, análise de custo-benefício e estratégias de retenção baseadas em perfis de uso.
